In [3]:
import cv2
import numpy as np
import os
from datetime import datetime

# ============================================================
# CẤU HÌNH ĐƯỜNG DẪN
# ============================================================
INPUT_FOLDER = r'D:\ADMIN\Documents\Classwork\advance_cv_project\sample'
OUTPUT_FOLDER = r'D:\ADMIN\Documents\Classwork\advance_cv_project\output_results'
LOG_FILE = r'processing_log.txt'

# ============================================================
# HÀM TOÁN HỌC (GIỮ NGUYÊN)
# ============================================================
def order_points(pts):
    rect = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect

def four_point_transform(image, pts):
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    dst = np.array([[0, 0], [maxWidth - 1, 0], [maxWidth - 1, maxHeight - 1], [0, maxHeight - 1]], dtype="float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    return cv2.warpPerspective(image, M, (maxWidth, maxHeight))

# ============================================================
# HÀM XỬ LÝ 1 ẢNH
# ============================================================
def process_single_image(img_path, save_dir):
    try:
        # 1. Đọc ảnh
        image = cv2.imread(img_path)
        if image is None:
            return False, "Lỗi đọc file"

        orig = image.copy()

        # 2. Resize
        ratio = image.shape[0] / 500.0
        image = cv2.resize(image, (int(image.shape[1]/ratio), 500))

        # 3. Tìm vùng giấy (Threshold 160)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)

        ret, thresh = cv2.threshold(blurred, 160, 255, cv2.THRESH_BINARY)
        thresh = cv2.erode(thresh, None, iterations=2)
        thresh = cv2.dilate(thresh, None, iterations=2)

        # 4. Tìm Contour
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]

        if len(cnts) == 0:
            return False, "Không tìm thấy contour"

        # Lấy contour lớn nhất -> MinAreaRect
        c = max(cnts, key=cv2.contourArea)
        if cv2.contourArea(c) < 1000:
             return False, "Vùng sáng quá nhỏ (Nhiễu)"

        rect = cv2.minAreaRect(c)
        box = cv2.boxPoints(rect)
        box = np.int0(box)

        # 5. Cắt ảnh
        warped = four_point_transform(orig, box.reshape(4, 2) * ratio)

        # 6. Xử lý OCR (Đen trắng)
        warped_gray = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
        warped_gray = cv2.convertScaleAbs(warped_gray, alpha=1.5, beta=0) # Tăng tương phản

        warped_bin = cv2.adaptiveThreshold(warped_gray, 255,
                                         cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                         cv2.THRESH_BINARY, 21, 10)

        # 7. CHỈ LƯU ẢNH OCR
        filename = os.path.basename(img_path)
        name, ext = os.path.splitext(filename)

        # Đặt tên file kết quả (ví dụ: image_01_ocr.jpg)
        save_path_ocr = os.path.join(save_dir, f"{name}_ocr.jpg")

        cv2.imwrite(save_path_ocr, warped_bin)

        return True, "Thành công"

    except Exception as e:
        return False, f"Lỗi ngoại lệ: {str(e)}"

# ============================================================
# CHƯƠNG TRÌNH CHÍNH
# ============================================================
def main():
    if not os.path.exists(INPUT_FOLDER):
        print(f"LỖI: Không tìm thấy thư mục {INPUT_FOLDER}")
        return

    if not os.path.exists(OUTPUT_FOLDER):
        os.makedirs(OUTPUT_FOLDER)

    valid_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".tiff"}
    files = [f for f in os.listdir(INPUT_FOLDER) if os.path.splitext(f)[1].lower() in valid_extensions]

    print(f"--- BẮT ĐẦU XỬ LÝ {len(files)} ẢNH ---")

    with open(LOG_FILE, "w", encoding="utf-8") as log:
        log.write(f"SESSION: {datetime.now()}\n---------------------\n")
        success = 0
        fail = 0

        for i, filename in enumerate(files):
            print(f"[{i+1}/{len(files)}] {filename}...", end=" ")

            status, msg = process_single_image(os.path.join(INPUT_FOLDER, filename), OUTPUT_FOLDER)

            log.write(f"{filename} | {'OK' if status else 'FAIL'} | {msg}\n")

            if status:
                print("-> OK")
                success += 1
            else:
                print(f"-> FAIL ({msg})")
                fail += 1

        print(f"\nHOÀN TẤT: {success} OK, {fail} Fail. Check log tại {LOG_FILE}")

if __name__ == "__main__":
    main()

--- BẮT ĐẦU XỬ LÝ 5 ẢNH ---
[1/5] image_01.jpg... -> OK
[2/5] image_140.jpg... -> OK
[3/5] image_65.jpg... -> OK
[4/5] image_69.jpg... -> OK
[5/5] zz.jpg... -> OK

HOÀN TẤT: 5 OK, 0 Fail. Check log tại processing_log.txt


C:\Users\OS\AppData\Local\Temp\ipykernel_20256\3852695682.py:77: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)
